In [2]:
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
import torch
from torch.utils.data import Dataset
import torch.nn.functional as F

# ✅ Set device (MPS if available)
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# 1. Load dataset (RAG-style question-context pairs)
dataset = load_dataset("neural-bridge/rag-dataset-12000", split="train")
pairs = [(q, c) for q, c in zip(dataset["question"], dataset["context"]) if isinstance(q, str) and q.strip() and isinstance(c, str) and c.strip()]

# 2. Tokenizer + model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModel.from_pretrained(model_name).to(device)

# 3. Apply LoRA
lora_config = LoraConfig(
    task_type=TaskType.FEATURE_EXTRACTION,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(base_model, lora_config)
model.to(device)
model.print_trainable_parameters()

# 4. Prepare dataset class
class RagPairDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        q, c = self.pairs[idx]
        q_tok = tokenizer(q, truncation=True, padding="max_length", max_length=128, return_tensors="pt")
        c_tok = tokenizer(c, truncation=True, padding="max_length", max_length=128, return_tensors="pt")
        return {
            "query_input_ids": q_tok["input_ids"].squeeze(0),
            "query_attention_mask": q_tok["attention_mask"].squeeze(0),
            "ctx_input_ids": c_tok["input_ids"].squeeze(0),
            "ctx_attention_mask": c_tok["attention_mask"].squeeze(0)
        }

train_dataset = RagPairDataset(pairs)

# 5. Contrastive loss (MultipleNegativesRankingLoss style)
def collate_fn(batch):
    q_ids = torch.stack([b["query_input_ids"] for b in batch])
    q_mask = torch.stack([b["query_attention_mask"] for b in batch])
    c_ids = torch.stack([b["ctx_input_ids"] for b in batch])
    c_mask = torch.stack([b["ctx_attention_mask"] for b in batch])
    return {"q_ids": q_ids.to(device), "q_mask": q_mask.to(device), "c_ids": c_ids.to(device), "c_mask": c_mask.to(device)}

# 6. Trainer wrapper
class LoRARetriever(torch.nn.Module):
    def __init__(self, base):
        super().__init__()
        self.model = base

    def forward(self, q_ids, q_mask, c_ids, c_mask):
        q_emb = self.model(input_ids=q_ids, attention_mask=q_mask).last_hidden_state[:, 0]  # [CLS]
        c_emb = self.model(input_ids=c_ids, attention_mask=c_mask).last_hidden_state[:, 0]
        scores = torch.matmul(q_emb, c_emb.T)
        labels = torch.arange(q_emb.size(0)).to(q_emb.device)
        loss = F.cross_entropy(scores, labels)
        return {"loss": loss}

lora_wrapper = LoRARetriever(model).to(device)

# 7. TrainingArguments
args = TrainingArguments(
    output_dir="lora_finetune_retriever",
    per_device_train_batch_size=16,
    learning_rate=5e-5,
    num_train_epochs=3,
    logging_steps=20,
    save_strategy="epoch",
    remove_unused_columns=False
)

# 8. HuggingFace Trainer
trainer = Trainer(
    model=lora_wrapper,
    args=args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=collate_fn
)

# 9. Train
trainer.train()

# 10. Save
model.save_pretrained("lora_finetune_retriever")
tokenizer.save_pretrained("lora_finetune_retriever")

print("LoRA fine-tuning complete. Model saved.")


✅ Using device: mps
trainable params: 73,728 || all params: 22,786,944 || trainable%: 0.3236


/var/folders/st/sgk4ydvx7vjfs18cwxz8vm7r0000gn/T/ipykernel_72249/2274326885.py:90: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
                                        
  0%|          | 0/1800 [1:11:16<?, ?it/s]       

{'loss': 0.6503, 'grad_norm': 0.6206285357475281, 'learning_rate': 4.9444444444444446e-05, 'epoch': 0.03}


                                          
  0%|          | 0/1800 [1:11:19<?, ?it/s]       

{'loss': 0.5296, 'grad_norm': 1.4012823104858398, 'learning_rate': 4.888888888888889e-05, 'epoch': 0.07}


...

{'loss': 0.2505, 'grad_norm': 2.391418695449829, 'learning_rate': 5.555555555555556e-07, 'epoch': 2.97}


                                          
  0%|          | 0/1800 [1:15:52<?, ?it/s]         

{'loss': 0.2658, 'grad_norm': 0.6648826003074646, 'learning_rate': 0.0, 'epoch': 3.0}


                                          
100%|██████████| 1800/1800 [04:42<00:00,  6.38it/s]


{'train_runtime': 282.3412, 'train_samples_per_second': 101.983, 'train_steps_per_second': 6.375, 'train_loss': 0.31293446328904895, 'epoch': 3.0}
LoRA fine-tuning complete. Model saved.
